In [2]:
from LoopDataConverter import LoopConverter, InputData, SurveyName, Datatype, NtgsConfig
from osgeo import gdal, osr
import os 
from map2loop import Project#, logging
import map2loop
import tempfile
# logging.set_level('debug')

In [3]:
def create_raster(output_path, bbox, epsg, pixel_size, value=100):
    minx, miny, maxx, maxy = bbox
    cols = int((maxx - minx) / pixel_size)
    rows = int((maxy - miny) / pixel_size)
    driver = gdal.GetDriverByName('GTiff')
    out_raster = driver.Create(output_path, cols, rows, 1, gdal.GDT_Byte)
    out_raster.SetGeoTransform([minx, pixel_size, 0, maxy, 0, -pixel_size])
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(epsg)
    out_raster.SetProjection(srs.ExportToWkt())
    out_band = out_raster.GetRasterBand(1)
    out_band.Fill(value)
    out_band.FlushCache()
    out_raster = None

In [4]:
xmin, ymax = 194970,7342378
xmax, ymin = 348104,7306552
zmin, zmax = -3000, 3000
bounding_box = {
    "minx": xmin,
    "miny": ymin,
    "maxx": xmax,
    "maxy": ymax,
    "base": zmin,
    "top": zmax,
}
f_path = "/home/rabii/Git_Repos/LoopDataConverter/test_data/NTGS_data/Henbury"
create_raster(
    os.path.join(f_path, "DEM.tif"),
    (bounding_box['minx'], bounding_box['miny'], bounding_box['maxx'], bounding_box['maxy']),
    7854,
    1000,
)

/home/rabii/Git_Repos/m2l_pypi/lib/python3.11/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [ ]:
xmin, ymax = 449200.89,7322334.93
xmax, ymin = 463912.6,7315529.2
zmin, zmax = -3000, 3000
bounding_box = {
    "minx": xmin,
    "miny": ymin,
    "maxx": xmax,
    "maxy": ymax,
    "base": zmin,
    "top": zmax,
}
f_path = "/home/rabii/Git_Repos/LoopDataConverter/test_data/NTGS_data"
create_raster(
    os.path.join(f_path, "DEM.tif"),
    (bounding_box['minx'], bounding_box['miny'], bounding_box['maxx'], bounding_box['maxy']),
    7854,
    1000,
)

In [ ]:
xmin, ymax = 449200.89,7322334.93
xmax, ymin = 463912.6,7315529.2
zmin, zmax = -3000, 3000
bbox_3d = {
    "minx": xmin,
    "miny": ymin,
    "maxx": xmax,
    "maxy": ymax,
    "base": zmin,
    "top": zmax,
}

xmin, ymax = -24.210915,134.499656
xmax, ymin = -24.272464,134.644287
zmin, zmax = -3000, 3000
bbox_3d = {
    "minx": xmin,
    "miny": ymin,
    "maxx": xmax,
    "maxy": ymax,
    "base": zmin,
    "top": zmax,
}

In [5]:
input_data = InputData(
    GEOLOGY="../test_data/NTGS_data/Henbury/HB_LithOutcrop_250K.shp", 
    STRUCTURE="../test_data/NTGS_data/Henbury/HB_StructData_250K.shp",
    FAULT="../test_data/NTGS_data/Henbury/HB_Fault_250K.shp",
    FOLD="../test_data/NTGS_data/Henbury/HB_Fold_250K.shp"
)

loop_converter = LoopConverter(survey_name=SurveyName.NTGS,
                               data=input_data)
loop_converter.convert()

In [ ]:
import numpy
loop_converter.data[Datatype.FAULT].loc[loop_converter.data[Datatype.FAULT]["Dip"] == 45]

In [6]:
path = tempfile.mkdtemp()
loop_converter.save(Datatype.GEOLOGY, os.path.join(path, "geology.shp"), file_extension="shp")
loop_converter.save(Datatype.STRUCTURE, os.path.join(path, "structures.shp"), file_extension="shp")
loop_converter.save(Datatype.FOLD, os.path.join(path, "folds.shp"), file_extension="shp")
loop_converter.save(Datatype.FAULT, os.path.join(path, "faults.shp"), file_extension="shp")

In [ ]:
import os
from map2loop.project import Project
from map2loop.m2l_enums import VerboseLevel
from map2loop.m2l_enums import Datatype
from map2loop.sampler import SamplerSpacing, SamplerDecimator
from map2loop.sorter import SorterUseHint, SorterUseNetworkX, SorterAgeBased, SorterAlpha
from map2loop.thickness_calculator import InterpolatedStructure, StructuralPoint
import time
from datetime import datetime
nowtime=datetime.now().isoformat(timespec='minutes')
model_name=nowtime.replace(":","-").replace("T","-")
loop_project_filename = os.path.join(model_name, "HB_map.loop3d")

t0 = time.time()

ntgs_config = NtgsConfig()
config = ntgs_config.config_map
# Specify the boundary of the region of interest in the appropriate projection coordinates
xmin, ymax = 194970,7342378
xmax, ymin = 348104,7306552
zmin, zmax = -3000, 3000
bounding_box = {
    "minx": xmin,
    "miny": ymin,
    "maxx": xmax,
    "maxy": ymax,
    "base": zmin,
    "top": zmax,
}
# Initialise the project with the shapefiles, dtm, config file
# output locations and projection to work in
proj = Project(
    geology_filename = os.path.join(path, "geology.shp"),
    # fault_filename = os.path.join(path, "faults.shp"),
    structure_filename = os.path.join(path, "structures.shp"),
    fold_filename= os.path.join(path, "folds.shp"),
    dtm_filename = "../test_data/NTGS_data/Henbury/DEM.tif",
    config_dictionary= config,
    clut_filename = '../test_data/NTGS_data/500kibg_colours.csv',
    clut_file_legacy = True,
    verbose_level = VerboseLevel.NONE,
    tmp_path = model_name,
    working_projection = "EPSG:28353",
    bounding_box = bounding_box,
    loop_project_filename = loop_project_filename,
    overwrite_loopprojectfile = True
)

# Remove faults less than 5km
proj.set_minimum_fault_length(1000.0)

# Set sampling distance for geology and fault maps to 200m
proj.set_sampler(Datatype.GEOLOGY, SamplerSpacing(200.0))
proj.set_sampler(Datatype.FAULT, SamplerSpacing(1000.0))

# Set to only take every second orientation observation (0 or 1 means take all observations)
proj.set_sampler(Datatype.STRUCTURE, SamplerDecimator(2))

# Set what text is expected for intrusions (contained within the description field)
# proj.map_data.config.geology_config["intrusive_text"] = "mafic intrusive"

# Set specific layers from the geology map to be ignored (commonly "cover" or "water")
# proj.set_ignore_codes(["cover", "Fortescue_Group", "A_FO_od"])

proj.set_thickness_calculator(InterpolatedStructure())
# Specify which stratigraphic columns sorter to use, other options are
# (SorterAlpha, SorterAgeBased, SorterUseHint, SorterUseNetworkX, SorterMaximiseContacts, SorterObservationProjections)
proj.set_sorter(SorterAlpha())

# Or you can run map2loop and pre-specify the stratigraphic column
# column = [
#     # youngest
#     'No_formal_name',
#     'Chandler_Limestone',
#     'Arumbera_Sandstone',
#     'Pertatataka_Formation',
#     'Waldo_Pedlar_Formation',
#     'Aralka_Formation',
#     'Areyonga_Formation',
#     'Bitter_Springs_Formation'
#     # oldest
# ]

# proj.run_all(user_defined_stratigraphic_column=column)
# Or you can get map2loop to run all column sorting algorithms it has and takes the one
# that has the longest total basal contact length
#proj.run_all(take_best=True)
proj.run_all()

t1 = time.time()

Datatype FAULT is not set and so cannot be loaded

Datatype FAULT_ORIENTATION is not set and so cannot be loaded

Datatype FAULT is not set and so cannot be loaded




/home/rabii/Git_Repos/m2l_pypi/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/rabii/Git_Repos/map2loop/map2loop/thickness_calculator.py:361: RuntimeWarning: Mean of empty slice
  mean = numpy.nanmean(_thickness)
/home/rabii/Git_Repos/m2l_pypi/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:2053: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/rabii/Git_Repos/m2l_pypi/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/rabii/Git_Repos/map2loop/map2loop/thickness_calculator.py:361: RuntimeWarning: Mean of empty slice
  mean = numpy.nanmean(_thickness)
/home/rabii/Git_Repos/m2l_pypi/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:2053: RuntimeWarning: Degrees of f

Cannot calculate thickness between Areyonga_Formation and Wallara_Formation


In [16]:
proj.stratigraphic_column.stratigraphicUnits

,layerId,name,minAge,maxAge,group,supergroup,ThicknessMean,ThicknessMedian,ThicknessStdDev,Order,code,colour
0,17,Petermann_Sandstone,-9999.0,-9999.0,Pertaoorrta_Group,,-1.000000,-1.000000,0.000000,0,NaN,#5d7e60
1,5,Goyder_Formation,-9999.0,-9999.0,Pertaoorrta_Group,,3668.940167,3589.297676,2127.527909,1,NaN,#cffb86
2,4,Deception_Formation,-9999.0,-9999.0,Pertaoorrta_Group,,1933.687423,2208.698147,842.611740,2,NaN,#1932e2
3,8,Illara_Sandstone,-9999.0,-9999.0,Pertaoorrta_Group,,2598.279283,1892.357870,2365.236767,3,NaN,#f48b70
4,20,Tempe_Formation,-9999.0,-9999.0,Pertaoorrta_Group,,913.773037,990.946677,238.283383,4,NaN,#e7f2f3
5,12,Namatjira_Formation,-9999.0,-9999.0,Pertaoorrta_Group,,3551.250897,3492.853066,1170.594319,5,NaN,#628304
6,3,Chandler_Formation,-9999.0,-9999.0,Pertaoorrta_Group,,23124.380342,23026.421580,345.009023,6,NaN,#16c432
7,1,Arumbera_Sandstone,-9999.0,-9999.0,Pertaoorrta_Group,,NaN,NaN,NaN,7,NaN,#106e8a
8,16,Pertatataka_Formation,-9999.0,-9999.0,None,,NaN,NaN,NaN,8,NaN,#0c2562
9,13,No_formal_name,-9999.0,-9999.0,None,,5890.094186,6250.486236,2480.650442,9,NaN,#387866


In [ ]:
import LoopProjectFile as LPF
from loopstructuralvisualisation import Loop3DView
from LoopStructural.modelling.input.project_file import LoopProjectfileProcessor as LPFProcessor
import LoopStructural
import numpy as np
from scipy.interpolate import RegularGridInterpolator
from osgeo import gdal
import pandas as pd

t2 = time.time()

LPFilename = loop_project_filename
fault_params = {
    'interpolatortype':'FDI',
    'nelements':1e4,
}
foliation_params = {
    'interpolatortype':'FDI' , # 'interpolatortype':'PLI',
    'nelements':1e5,  # how many tetras/voxels
    'regularisation':5,
}

projFile = LPF.ProjectFile(LPFilename)
processedData = LPFProcessor(projFile)
processedData.foliation_properties['sg'] = foliation_params
# processedData.fault_properties['interpolatortype'] = fault_params['interpolatortype']
# processedData.fault_properties['nelements'] = fault_params['nelements']

model = LoopStructural.GeologicalModel.from_processor(processedData)
# model.nsteps=np.array([200,200,50])
model.update()

# clip_on_dtm=True
# if(clip_on_dtm):
#     bounding_box = proj.map_data.get_bounding_box()
#     model_base = bounding_box['base']
#     model_top = bounding_box['top']
#     dtm = gdal.Open('./source_data/dtm_rp.tif')
#     dtm_val = dtm.GetRasterBand(1).ReadAsArray().T
#     geoTrans = dtm.GetGeoTransform()
#     minx = geoTrans[0]
#     maxx = minx + dtm.RasterXSize * geoTrans[1]
#     miny = geoTrans[3]
#     maxy = miny + dtm.RasterYSize * geoTrans[5]
# 
#     # Convert bounds to gdal raster bounds
#     x = np.linspace(minx,maxx,dtm.RasterXSize)
#     y = np.linspace(miny,maxy,dtm.RasterYSize)
#     dtm_interpolator = RegularGridInterpolator((x,y),dtm_val)
#     model.dtm = lambda xyz : dtm_interpolator(xyz[:,:2])

vtk_path = os.path.join(model_name,'vtk')
if not os.path.exists(vtk_path):
    os.mkdir(vtk_path)
filename = os.path.join(model_name,'vtk','surface_name_{}.vtk')
view = Loop3DView(model)
view.nsteps=np.array([500,500,50])
for sg in model.feature_name_index:
    if( 'super' in sg):
        view.add_data(model.features[model.feature_name_index[sg]])
view.nelements = 1e5
view.add_model_surfaces(filename=filename,faults=False)
view.nelements=1e6
view.add_model_surfaces(filename=filename,strati=False,displacement_cmap = 'rainbow')
view.lv.webgl(vtk_path+model_name)
view.nsteps = np.array([500,500,50])
if(clip_on_dtm):
    colours = list(pd.DataFrame(data=proj.stratigraphic_column.column,columns=["name"]).merge(proj.stratigraphic_column.stratigraphicUnits[["name","colour"]], on="name")["colour"])
    colours.reverse()
    view.add_dtm(paint_with=lambda xyz: model.evaluate_model(xyz,scale=False), cmap=colours)
view.add_model()

# view.lv.control.Range('alpha', label="Global Opacity")
# view.lv.control.DualRange(['xmin', 'xmax'], label="x clip", step=0.01, values=[0.0,1.0])
# view.lv.control.DualRange(['ymin', 'ymax'], label="y clip", step=0.01, values=[0.0,1.0])
# view.lv.control.DualRange(['zmin', 'zmax'], label="z clip", step=0.01, values=[0.0,1.0])
# view.lv.control.Range(command='background', range=(0,1), step=0.1, value=0.8)
# view.lv.control.show() #Show the control panel, including the viewer window
# view.interactive()

t3 = time.time()